In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate, Input, Flatten, LeakyReLU,BatchNormalization
from keras.layers import Multiply

from keras.models import Model
from keras import optimizers
from keras import regularizers
import json
import re
import math

Using TensorFlow backend.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pro

In [2]:
def jsons_name_type(jsons_name):
    if not jsons_name:
        return -1
    if "Цифра дня" in jsons_name:
        return 1
    if "AliExpress" in jsons_name:
        return 2 
    if "Тинькофф" in jsons_name or "Tinkoff" in jsons_name:
        return 3
    if "ЧМ" in jsons_name:
        return 4
    if "фильм"  in jsons_name or "смотрет"  in jsons_name:
        return 5
    if "кредит"  in jsons_name:
        return 6
    if "инвестиц"  in jsons_name:
        return 7
    if "₽"  in jsons_name or "$"  in jsons_name:
        return 8
    if "тпуск"  in jsons_name:
        return 9
    return 0

In [3]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_train['first_session_dttm']=pd.to_datetime(customer_train['first_session_dttm'],infer_datetime_format=True)
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
customer_test['first_session_dttm']=pd.to_datetime(customer_test['first_session_dttm'],infer_datetime_format=True)
stories_description = pd.read_csv(f'{data_path}stories_description.csv')

stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [4]:

subsets = stories_reaction_train.groupby(['customer_id', 'event_dttm']).count().reset_index()[['customer_id', 'event_dttm', 'story_id']]
subsets.columns = ['customer_id', 'event_dttm','subsets']
stories_reaction_train=stories_reaction_train.merge(subsets, on=['customer_id', 'event_dttm'], how='left')

subsets = stories_reaction_test.groupby(['customer_id', 'event_dttm']).count().reset_index()[['customer_id', 'event_dttm', 'story_id']]
subsets.columns = ['customer_id', 'event_dttm','subsets']
stories_reaction_test=stories_reaction_test.merge(subsets, on=['customer_id', 'event_dttm'], how='left')

In [5]:
stories_reaction_train['reaction_hour']=stories_reaction_train.event_dttm.dt.hour
stories_reaction_test['reaction_hour']=stories_reaction_test.event_dttm.dt.hour
stories_reaction_train['reaction_dayofweek']=stories_reaction_train.event_dttm.dt.dayofweek
stories_reaction_test['reaction_dayofweek']=stories_reaction_test.event_dttm.dt.dayofweek

stories_description['jsons'] = stories_description.story_json.map(lambda x:json.loads(x.replace("\\\\", "\\")))
stories_description['jsons_name'] =stories_description.jsons.map(lambda x:x['properties'].get('name'))
stories_description['texts'] =stories_description.jsons.map(lambda x:re.findall(""""text": \"([^\"]*)\",""", json.dumps(x, ensure_ascii=False)))
stories_description['jsons_name_type'] = stories_description.jsons_name.map(lambda x:jsons_name_type(x))

In [6]:
customer_train['first_session_dttm_year']=(customer_train.first_session_dttm.dt.year-2000)/20
customer_test['first_session_dttm_year']=(customer_test.first_session_dttm.dt.year-2000)/20


In [7]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
train_and_val = train_and_val.merge(stories_description, on='story_id', how='left')
test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')
test = test.merge(stories_description, on='story_id', how='left')
train_and_val=train_and_val.fillna(0)
test=test.fillna(0)

In [8]:
# stories_history = train_and_val.sort_values('event_dttm')[['customer_id', 'story_id']].groupby('customer_id').story_id.apply(list).to_frame().reset_index()
# stories_history.columns = ['customer_id', 'stories_history']
# train_and_val = train_and_val.merge(stories_history, on='customer_id', how='left')
# train_and_val[['stories_history', 'story_id']].stories_history.values[0].index(138)
# train_and_val['stories_history2'] = train_and_val[['stories_history', 'story_id']].apply(lambda x : x['stories_history'][:x['stories_history'].index(x['story_id'])],axis=1)
# def static_history(x, size):
#     res = ([0]*max(0,(size-len(x))))
#     res.extend(x[-size:])
#     return res
# train_and_val['stories_history3']=train_and_val.stories_history2.map(lambda x :static_history(x,10))

In [9]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())

val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]


# seed = 4
# np.random.seed(seed)
# msk = np.random.rand(len(train_and_val)) < 0.8
# train = train_and_val[msk]
# val = train_and_val[~msk]


In [10]:
# customer_shows = train.groupby('customer_id')['event'].count().to_frame().reset_index() 
# customer_shows.columns = ['customer_id', 'customer_shows']
# customer_shows=customer_shows.fillna(0)
# train=train.merge(customer_shows, on='customer_id')
# val=val.merge(customer_shows, on='customer_id', how='left')
# train['customer_shows']=train.customer_shows.fillna(-999)
# val['customer_shows']=val.customer_shows.fillna(-999)

In [11]:
len(train), len(val) , len(test) 

(320248, 152893, 172049)

In [12]:

train['gender_cd']=train.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
val['gender_cd']=val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
test['gender_cd']=test.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))

train['children_cnt']=train.children_cnt/10
val['children_cnt']=val.children_cnt/10
test['children_cnt']=test.children_cnt/10



/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [14]:
len(range(1, 10+1))

10

In [15]:
all_data = pd.concat([train_and_val, test])
def to_OHE(field):
    count = len(set(all_data[field].values))
    d = dict(zip(list(set(all_data[field].values)), range(1, count+1)))
    train[field]=train[field].map(lambda x:  d[x])
    val[field]=val[field].map(lambda x:  d[x])
    test[field]=test[field].map(lambda x:  d[x])
    return count, d
customers_count, customer_id_dictd = to_OHE('customer_id')
stories_count, _ = to_OHE('story_id')
jsons_name_type_count, _ = to_OHE('jsons_name_type')




/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [16]:
train

,customer_id,story_id,event_dttm,event,subsets,reaction_hour,reaction_dayofweek,product_0,product_1,product_2,...,children_cnt,first_session_dttm,job_position_cd,job_title,first_session_dttm_year,story_json,jsons,jsons_name,texts,jsons_name_type
1,8,31,2018-06-04 08:08:08,skip,7,8,0,0,CLS,UTL,...,0.1,2012-06-06 09:57:21,17.0,НАЧАЛЬНИК ОТДЕЛА,0.6,"{""guid"":""c4dfc7e9-c523-4790-b8a7-f649f2119340""...",{'guid': 'c4dfc7e9-c523-4790-b8a7-f649f2119340...,Скрытые карты,"[Скрытые счета и карты, Можно скрыть любую кар...",1
2,8,40,2018-06-17 13:44:45,skip,2,13,6,0,CLS,UTL,...,0.1,2012-06-06 09:57:21,17.0,НАЧАЛЬНИК ОТДЕЛА,0.6,"{""guid"":""106f9cb3-e246-4e6b-80f1-a74637884f2e""...",{'guid': '106f9cb3-e246-4e6b-80f1-a74637884f2e...,Помощь в чате,"[Помощь в чате, Необязательно звонить в банк, ...",1
3,8,118,2018-05-23 05:41:43,skip,4,5,2,0,CLS,UTL,...,0.1,2012-06-06 09:57:21,17.0,НАЧАЛЬНИК ОТДЕЛА,0.6,"{""guid"":""d04ad496-8783-4fc9-9789-39c939abe303""...",{'guid': 'd04ad496-8783-4fc9-9789-39c939abe303...,Налог с перевода,"[ФИНАНСЫ, Когда не нужно платить налог с перев...",1
7,8,291,2018-05-23 05:41:43,view,4,5,2,0,CLS,UTL,...,0.1,2012-06-06 09:57:21,17.0,НАЧАЛЬНИК ОТДЕЛА,0.6,"{""guid"":""fade1ca4-e0cd-4a2d-996b-1f0922ec07af""...",{'guid': 'fade1ca4-e0cd-4a2d-996b-1f0922ec07af...,Что читать на выходных,"[Что читать на выходных, Редакция «Букмейта» в...",1
8,8,343,2018-06-04 08:08:08,skip,7,8,0,0,CLS,UTL,...,0.1,2012-06-06 09:57:21,17.0,НАЧАЛЬНИК ОТДЕЛА,0.6,"{""guid"":""872f088f-2d42-4ff0-b7b9-2c98b350d1f6""...",{'guid': '872f088f-2d42-4ff0-b7b9-2c98b350d1f6...,Франшиза: кофе-бар,"[Как открыть кофе-бар по франшизе, Это Никита....",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473128,22147,319,2018-05-24 08:32:41,view,6,8,3,0,0,0,...,0.0,0,22.0,0,0.0,"{""guid"":""c25b7d2c-0f1e-4985-b1da-46042fdfe473""...",{'guid': 'c25b7d2c-0f1e-4985-b1da-46042fdfe473...,Простые игры для смартфона,"[Простые и интересные игры для смартфона, Мини...",1
473129,22147,325,2018-05-24 08:32:41,view,6,8,3,0,0,0,...,0.0,0,22.0,0,0.0,"{""guid"":""e8e5ff08-6942-482d-9852-9710e7104df6""...",{'guid': 'e8e5ff08-6942-482d-9852-9710e7104df6...,Вымогают деньги,"[Что делать, если мошенники вымогают деньги, З...",1
473130,22147,451,2018-06-18 09:14:46,view,4,9,0,0,0,0,...,0.0,0,22.0,0,0.0,"{""guid"":""6d8b893b-1eb3-4be4-bb7e-c6778db8da80""...",{'guid': '6d8b893b-1eb3-4be4-bb7e-c6778db8da80...,Проверьте себя,"[Допустим, у вас не прошел платеж по карте, Вы...",1
473131,22147,479,2018-05-24 08:32:41,view,6,8,3,0,0,0,...,0.0,0,22.0,0,0.0,"{""guid"":""9d4b77e4-a9e4-4a1c-8141-1a084cd342df""...",{'guid': '9d4b77e4-a9e4-4a1c-8141-1a084cd342df...,Проверьте себя,"[Допустим, вам приходит СМС, От имени ЦБ РФ ва...",1


In [17]:
val_test = pd.concat([val, test])
train = train.drop_duplicates(['customer_id', 'story_id', 'event'])
# train = train[train.customer_id.isin(set(val.customer_id.values))]
importance = val_test.customer_id.value_counts().map(lambda x : math.log(x+1)).to_frame().reset_index()
importance.columns = ['customer_id', 'importance']

train_importance = train.customer_id.value_counts().map(lambda x : math.log(x+1)).to_frame().reset_index()
train_importance.columns = ['customer_id', 'train_importance']

# importance = val.customer_id.value_counts().map(lambda x : x+1).to_frame().reset_index()
train = train.merge(importance, on='customer_id', how='left')
train = train.merge(train_importance, on='customer_id', how='left')

train['importance'].fillna(0.5, inplace=True)
train['train_importance'].fillna(0.5, inplace=True)

# train['importance'] = train[['importance','train_importance']].apply(lambda x: x['importance']/min(x['importance'],x['train_importance']), axis=1)
train['importance'] = train[['importance','train_importance']].apply(lambda x: x['importance']/max(x['importance'],x['train_importance']), axis=1)



/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
# values = set(np.concatenate(np.array(train_and_val.stories_history3.values)))
# stories_history_count = len(values)
# d = dict(zip(list(values), range(1, len(values)+1)))

# train['stories_history3'] = train['stories_history3'].map(lambda f:  [d[i] for i in f])
# val['stories_history3'] = val['stories_history3'].map(lambda f:  [d[i] for i in f])

In [19]:
values = set(all_data[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values.reshape(-1))
producs_count = len(values)
d = dict(zip(list(values), range(1, len(values)+1)))
for f in ['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']:
    train[f]=train[f].map(lambda x:  d[x])
    val[f]=val[f].map(lambda x:  d[x])
    test[f]=test[f].map(lambda x:  d[x])

/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
import keras.backend as K
def metric_loss(batch_size):
    d = np.sum(np.abs(valY))
    def the_loss(y_true, y_pred):
        return -(len(val.customer_id.values)/batch_size)*K.sum(y_true*y_pred)/d
    return(the_loss)


In [21]:
def reaction_to_class(reaction, importance=1):
    if reaction=="dislike":
        return -10*importance
    if reaction=="like":
        return 0.5*importance
    if reaction=="view":
        return 0.1*importance
    if reaction=="skip":
        return -0.1*importance
trainY = np.array(list(train[['event', 'importance']].apply(lambda x: reaction_to_class(x['event'], x['importance']),axis=1).values))
valY = np.array(list(val.event.map(lambda x: reaction_to_class(x)).values))

In [22]:
trainY

array([-0.1, -0.1, -0.1, ...,  0.1,  0.1,  0.1])

In [23]:
valY

array([ 0.1,  0.1,  0.1, ..., -0.1,  0.1, -0.1])

In [24]:

model_input_customer = Input((1,), name='customer_id')
customer = Embedding(customers_count+1, 1, input_length=1)(model_input_customer)
customer = Flatten()(customer)

model_input_story = Input((1,), name='story_id')
story = Embedding(stories_count, 4, input_length=1)(model_input_story)
story = Flatten()(story)

model_input_products = Input((7,), name='producs')
products = Embedding(producs_count, 1, input_length=7)(model_input_products)
products = Flatten()(products)



model_input_story_name = Input((1,), name='story_name_id')
story_name = Embedding(jsons_name_type_count, 2, input_length=1)(model_input_story_name)
story_name = Flatten()(story_name)

model_input_subset = Input((1,), name='model_input_subset')
# subset = Embedding(20, 2, input_length=1)(model_input_subset)
# subset = Flatten()(subset)


model_input_3 = Input((3,), name='customer_data')
model_input_4 = Input((2,), name='customer_data2')


x_other = Concatenate()([ products, story_name, model_input_3, model_input_4, model_input_subset])
x = Concatenate()([customer, story])


x = Dense(16,
          activation='relu', 
         )(x)


x_other = Dense(8 ,
          activation='relu', 
         )(x_other)





x = Concatenate()([x, x_other])



x = Dense(1, activation='tanh')(x)
model = Model([ 
    model_input_customer, 
               model_input_story, 
               model_input_story_name,
               model_input_subset,
               model_input_3,
               model_input_4, 
               model_input_products,
              ], x)





print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
customer_id (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
story_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
producs (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
story_name_id (InputLayer)      (None, 1)            0                                            
____________________________________________________________________________________________

In [25]:
# batch_size=320248
batch_size=32*4
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss=metric_loss(batch_size))


In [26]:


model.fit(
        x = [
#             np.stack(np.array(train.stories_history3.values)),
            
            train.customer_id.values,
             train.story_id.values,
             train.jsons_name_type.values, 
             train.subsets.values/20,
             
             train[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
             train[['reaction_hour', 'reaction_dayofweek']].values/24,
             train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
            ], 
          y = trainY,
          batch_size=batch_size,
          epochs=1,
          validation_data=([
#               np.stack(np.array(val.stories_history3.values)),
              
              val.customer_id.values, 
                            val.story_id.values, 
                            val.jsons_name_type.values, 
                            val.subsets.values/20,
                            val[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
                            val[['reaction_hour', 'reaction_dayofweek']].values/24,
                            val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
                           ], valY),
         )

# 320248/320248 [==============================] - 13s 41us/step - loss: -0.6153 - val_loss: -0.3492
# 320248/320248 [==============================] - 13s 42us/step - loss: -0.7373 - val_loss: -0.3607
# 320248/320248 [==============================] - 14s 42us/step - loss: -0.7263 - val_loss: -0.3656 size 4 tanh
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.7046 - val_loss: -0.3612 size 128 relu
# 320248/320248 [==============================] - 14s 44us/step - loss: -0.7110 - val_loss: -0.3455 size 128 lrelu
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.6390 - val_loss: -0.3683 size 128 relu increase embs
# 320248/320248 [==============================] - 16s 48us/step - loss: -0.6319 - val_loss: -0.3705 job_position_cd


# 378578/378578 [==============================] - 61s 161us/step - loss: -0.4684 - val_loss: -0.4269
# 378578/378578 [==============================] - 15s 41us/step - loss: -0.5597 - val_loss: -0.4413
# 378578/378578 [==============================] - 2s 6us/step - loss: -0.5903 - val_loss: -0.4453


Train on 309664 samples, validate on 152893 samples
Epoch 1/1
309664/309664 [==============================] - 7s 23us/step - loss: -0.1922 - val_loss: -0.3243


In [27]:
customer_id_dict={y:x for x,y in customer_id_dictd.items()}
    

In [29]:
customer_id_dict

{1: 786434,
 2: 786435,
 3: 393220,
 4: 917511,
 5: 917512,
 6: 262156,
 7: 524301,
 8: 15,
 9: 393231,
 10: 917520,
 11: 393235,
 12: 786453,
 13: 786454,
 14: 262168,
 15: 524313,
 16: 917530,
 17: 393243,
 18: 262176,
 19: 917538,
 20: 262181,
 21: 917541,
 22: 262183,
 23: 262184,
 24: 524329,
 25: 786473,
 26: 917544,
 27: 393260,
 28: 131120,
 29: 131121,
 30: 524336,
 31: 131123,
 32: 786482,
 33: 262197,
 34: 786487,
 35: 131133,
 36: 655425,
 37: 524358,
 38: 655431,
 39: 131142,
 40: 73,
 41: 917578,
 42: 393294,
 43: 917584,
 44: 393298,
 45: 393300,
 46: 917594,
 47: 91,
 48: 524390,
 49: 393322,
 50: 786538,
 51: 786541,
 52: 262255,
 53: 786547,
 54: 655479,
 55: 131192,
 56: 917623,
 57: 655486,
 58: 129,
 59: 917633,
 60: 786572,
 61: 655501,
 62: 144,
 63: 150,
 64: 393366,
 65: 655510,
 66: 655511,
 67: 131227,
 68: 131228,
 69: 524443,
 70: 393374,
 71: 655515,
 72: 164,
 73: 262309,
 74: 786598,
 75: 393386,
 76: 131244,
 77: 262319,
 78: 524464,
 79: 178,
 80: 9176

In [30]:
cust_to_embs = {}
customer_id_dict[0]=0
for i, embs in enumerate(model.layers[4].get_weights()[0]):
    cust_to_embs[customer_id_dict[i]]=embs

In [31]:
cust_to_embs[131068]

array([0.01664245], dtype=float32)

In [32]:
np.save("cust_embs", cust_to_embs)

In [26]:
# train_predict = model.predict([train.customer_id.values,
#              train.story_id.values,
#              0*train.jsons_name_type.values, 
#              train.job_title.values,
#              train.job_position_cd.values,
#              0*train.marital_status_cd.values,
#              train[['age', 'gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
#              train[[ 'customer_shows']].values/3000,
#              0*train[['transaction_amt_count', 'transaction_amt_min', 'transaction_amt_max']].values,
#              train[['reaction_hour', 'reaction_dayofweek']].values/30,
#              train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
#            ])
val_predict = model.predict([
              
                            val.customer_id.values, 
                            val.story_id.values, 
                            val.jsons_name_type.values, 
                            val.subsets.values/20,
                            val[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
                            val[['reaction_hour', 'reaction_dayofweek']].values/24,
                            val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
                          ])
# train_predict = model.predict(train.customer_id.values)
# val_predict = model.predict(val.customer_id.values)
-np.sum(valY*val_predict[:,0] )/np.sum(np.abs(valY))
# -0.3488346709728467



KeyboardInterrupt: 

In [ ]:
test_predict = model.predict([
              
                            test.customer_id.values, 
                            test.story_id.values, 
                            test.jsons_name_type.values, 
                            test.subsets.values/20,
                            test[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
                            test[['reaction_hour', 'reaction_dayofweek']].values/24,
                            test[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
                          ])

test['score'] = test_predict
test[['answer_id', 'score']].to_csv('pred' + str(seed) + '.csv', index=False)

In [ ]:
# 1 0.4357
# 2 0.4192
# 3 0.4351